__собираем предложения о земельных участках__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/land/avito' 
# cian_raw_data_path = 'data/raw/cian'
result_file_path = 'data/data_land.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/land/avito/avito_2022-09-08_16-05_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-12_13-51_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-13_13-56_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-15_10-31_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-16_15-10_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-19_12-39_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-22_09-57_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-26_11-22_raw.xlsx',
 'data/raw/land/avito/avito_2022-10-08_20-38_raw.xlsx',
 'data/raw/land/avito/avito_2022-10-14_15-54_raw.xlsx',
 'data/raw/land/avito/avito_2022-11-01_13-58_raw.xlsx',
 'data/raw/land/avito/avito_2022-11-04_17-12_raw.xlsx',
 'data/raw/land/avito/avito_2022-11-07_10-46_raw.xlsx',
 'data/raw/land/avito/avito_2022-11-15_13-16_raw.xlsx',
 'data/raw/land/avito/avito_2022-11-28_14-58_raw.xlsx']

In [7]:
# for f in raw_data_files:
#     df = pd.read_excel(f)
#     df['place']='sevastopol'
#     df.to_excel(f,index=False)
#     print(f)

In [8]:
data_avito = pd.concat([
        pd
        .read_excel(f)
        #.dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

print(len(data_avito))

16199


In [9]:
# data_avito

In [10]:
from lib.avito import AvitoDataCleanerRealtyLand

data_avito = AvitoDataCleanerRealtyLand().transform( data_avito )

print(len(data_avito))

16199


In [11]:
data_avito.sample(3)

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,area,is_IJS,priceM,priceMU,area_size_category
15359,2662734907,Участок 4 сот. (ИЖС),1900000,,садоводческое товарищество Малахит,"Эксклюзивное предложение, цена низкая из за срочности, свет, вода,. Шикарный вид, удобное местоположение. Документы Россия. Отличное предложение для вас. Доп фото.\nСкину. Участок не один. При желании поможем в строительстве, из любых материалов.",4,2022-11-28 14:58:16.679,sevastopol,4.00,True,1.90,0.47,2-4
14943,2481317582,Участок 4 сот. (ИЖС),1600000,,товарищество собственников недвижимости Гидротехник,"Продам земельный участок 4 сотки в СТ Гидротехник. Участок ровный правильной формы, находится рядом с сосновым лесом. \n\nКоммуникации: электричество 3х фазное подключено, вода привозная или делать скважину (сейчас решается вопрос о городской воде). Подъездн",18,2022-11-15 13:16:46.237,sevastopol,4.00,True,1.60,0.40,2-4
7486,2245589563,"Участок 4,5 сот. (ИЖС)",3250000,,ТСН СТ Фиолент,"Продам земельный участок 4.5 сотки в закрытом СТ Фиолент. \n\nИнфраструктура: \n\n+ Пропускной режим в Садовое товарищество (на въезде встречает частное охранное предприятие, установлен шлагбаум с электро-пультом);\n\n+ Остановка общественного транспорта в 5 мин",18,2022-09-22 09:57:15.623,sevastopol,4.50,True,3.25,0.72,4-8


In [12]:
assert len(data_avito)>0

In [13]:
data_avito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16199 entries, 0 to 16198
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   avito_id            16199 non-null  int64         
 1   title               16199 non-null  object        
 2   price               16199 non-null  int64         
 3   obj_name            16199 non-null  object        
 4   adr                 16199 non-null  object        
 5   description         16199 non-null  object        
 6   avito_page          16199 non-null  int64         
 7   ts                  16199 non-null  datetime64[ns]
 8   place               16199 non-null  object        
 9   area                16199 non-null  float64       
 10  is_IJS              16199 non-null  bool          
 11  priceM              16199 non-null  float64       
 12  priceMU             16199 non-null  float64       
 13  area_size_category  16189 non-null  category  

---

In [14]:
data_avito['url'] = 'https://avito.ru/' + data_avito['avito_id'].astype(int).astype(str)
data_avito = data_avito.drop(columns=['avito_id'])
# data_cian = data_cian.rename(columns={'cian_url':'url'})

In [15]:
data = data_avito.reset_index(drop=True) 
print(len(data))
assert len(data)>0

16199


## обновляем таблицу адресов

In [16]:
# data_ = data.sample(15).reset_index(drop=True)
# data_[['adr']]

In [17]:
places = { 
# поле 'place' и название города
'bahchisaray': 'Бахчисарай,',
      'sudak': 'Судак,',
  'sevastopol': 'Севастополь,',
}

def norm_place(d,places=places):
    if not d['place'] in places: return d['adr_']
    p = places[ d['place'] ]
    return d['adr_'] if d['adr_'].find(p)>-1 else p+d['adr_']

In [18]:
from lib.locator import AddressTransformer

atr = AddressTransformer()
data['adr_'] = data['adr'].progress_apply(atr.transform)
data['adr_'] = data[['adr_','place']].progress_apply(norm_place,axis=1)
data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

  0%|          | 0/16199 [00:00<?, ?it/s]

  0%|          | 0/16199 [00:00<?, ?it/s]

In [19]:
data[['adr_orig','adr']].sample(10)

,adr_orig,adr
8724,"пос. городского типа Кача, ул. Авиаторов","Севастополь,пос. городского типа Кача, ул. Авиаторов"
7044,ул. 2-я Линия,"Севастополь,ул. 2-я Линия"
1359,"ул. Генерала Рихтера, 29","Севастополь,ул. Генерала Рихтера, 29"
7892,"ул. Генерала Жидилова, 49","Севастополь,ул. Генерала Жидилова, 49"
13674,"Тарутинская ул., 58А","Севастополь,Тарутинская ул., 58А"
3988,"Вербеновый пр., 5","Севастополь,Вербеновый пр., 5"
9982,"с. Широкое, ул. Микадзе, 14Б","Севастополь,с. Широкое, ул. Микадзе, 14Б"
5977,"с. Тыловое, Стахановская ул.","Севастополь,с. Тыловое, Стахановская ул."
8811,"ул. Челнокова, 47А","Севастополь,ул. Челнокова, 47А"
16124,"с. Угловое, ул. Гунько","Севастополь,с. Угловое, ул. Гунько"


In [20]:
# from lib.locator import GeocoderSimpleOSM
from lib.locator import GeocoderYandex
from yandex_keys import keys

atr = AddressTransformer()
locator = GeocoderYandex(keys[0])
# locator = GeocoderSimpleOSM()

loc_file_path = 'data/location.pkl' # таблица адресов
loc = pd.read_pickle(loc_file_path)
# loc = pd.DataFrame([],columns=['adr','latitude','longitude',])
logging.info(f'LocationUpdater: {len(loc)} addresses in index')

# собираем все адреса в один список
loc = loc.merge(
        data[['adr']].drop_duplicates(),
        how='outer',
        on='adr',
    ).drop_duplicates().reset_index(drop=True)

logging.info(f'LocationUpdater: {len(loc)} addresses total')

# собираем все адреса с геопозицией
loc_def = loc[~loc['latitude'].isna()].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_def)} addresses defined')

# собираем все адреса без геопозиции
loc_undef = loc[loc['latitude'].isna()][['adr']].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_undef)} addresses undefined')

if len(loc_undef)>0:
    # определяем координаты по строке адреса
    loc_undef = loc_undef.join( loc_undef['adr'].progress_apply(locator.transform).apply(pd.Series) ).dropna()     
    logging.info(f'LocationUpdater: {len(loc_undef)} new addresses found')
    loc = pd.concat([loc_def,loc_undef]).reset_index(drop=True) 
    assert len(loc)>0
    loc.to_pickle(loc_file_path)

[INFO    ] 2022-11-28 17:10:00 | LocationUpdater: 7420 addresses in index
[INFO    ] 2022-11-28 17:10:00 | LocationUpdater: 7481 addresses total
[INFO    ] 2022-11-28 17:10:00 | LocationUpdater: 7420 addresses defined
[INFO    ] 2022-11-28 17:10:00 | LocationUpdater: 61 addresses undefined


  0%|          | 0/61 [00:00<?, ?it/s]

[INFO    ] 2022-11-28 17:10:29 | LocationUpdater: 61 new addresses found


In [21]:
print( len(loc) )
loc.sample(3)

7481


,latitude,longitude,adr
1088,44.59,33.48,"Севастополь,Университетская улица , 20/24"
3065,44.58,33.58,"Севастополь,ул. Горпищенко"
1057,44.59,33.44,"Севастополь,Античный проспект , 9"


## дополняем данные геометкой

In [22]:
# data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

data = data.merge(loc,on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isna() ] ) )

assert len(data)>0
data.to_pickle(result_file_path)

data.info()

всего записей: 16199
записей без геометки: 0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16199 entries, 0 to 16198
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               16199 non-null  object        
 1   price               16199 non-null  int64         
 2   obj_name            16199 non-null  object        
 3   adr_orig            16199 non-null  object        
 4   description         16199 non-null  object        
 5   avito_page          16199 non-null  int64         
 6   ts                  16199 non-null  datetime64[ns]
 7   place               16199 non-null  object        
 8   area                16199 non-null  float64       
 9   is_IJS              16199 non-null  bool          
 10  priceM              16199 non-null  float64       
 11  priceMU             16199 non-null  float64       
 12  area_size_category  16189 non-null  category      
 13  u